# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Luis Picón
- Nombre de alumno 2: Israel Astudillo M.

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/IsraPKMNPAP/Laboratorio-de-Herramientas)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [1]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [2]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

El ambiente BlackJack es un ambiente de contexto de aprendizaje reforzado que impone las reglas y funcionamiento de la situación a simular, en este caso un juego de Black Jack.
La formulación MDP es la siguiente en base a las variables que incluye el modelo:
- Acciones: Hay dos posibles acciones realizables por el agente las cuales son "stick" o "hit" que representan quedarse con las cartas actuales o pedir una carta extra respectivamente. Está descrito formalmente como Discrete(2) dado que hay 2 acciones.
- Estados: La información que obtiene el agente de cada estado en el tiempo y que por tanto describen cada estado son la suma total de las cartas que posee el jugador, el valor de la carta del dealer boca arriba y si el jugador posee o no un as usable o que puede cambiar su valor. Descrito formalmente como Tuple(Discrete(32), Discrete(11), Discrete(2)) dado que la suma de cartas del jugador puede tomar 32 valores, la del dealer 11 valores y la si tiene un as dos valores. Así, hay 32 x 11 x 2 = 704 posibles estados que puede tener el espacio de observación.
- Recompensas: Las recompensas entregadas son +1 si es que se gana el juego, -1 si se pierde, 0 si se empata y 1.5 si el blackjack de la victoria es natural.

#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [3]:
# Recompensas
R = []
# Loop de simulaciones
for episode in range(5000):
  # Nueva iteración, reseteamos el juego
  obs = env.reset()
  # Inicializamos variable de término del juego
  done = False
  # Loop de cada juego, utilizamos la variable de término del juego
  while not done:
    # Acción aleatoria
    action = env.action_space.sample()
    # Resultado de la acción
    obs, reward, done,truncated, info = env.step(action)
    # Continúa hasta que el juego termine, cuando done=True
  # Terminado el juego actual, reportamos la recompensa
  R.append(reward)
  # Siguiente juego

In [4]:
print("Promedio de recompensas: ", np.mean(R))
print("Desviación de recompensas: ", np.std(R))

Promedio de recompensas:  -0.3894
Desviación de recompensas:  0.8994262838054045


In [5]:
env.close()

El performance parece ser malo, dado que su promedio de ganancias es negativo.
El performance de la política aleatoria tiene un valor promedio negativo, lo cual nos indica que en promedio se pierde más de lo que se gana. Por otro lado, la desviación de las recompensas es bastante alta, cercana a ser la unidad completa que se asigna de premio o castigo en el juego. En general las recompensas obtenidas con esta política no llevan a un desempeño promedio positivo.

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [6]:
# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

In [7]:
# Usamos PPO que soporta todos los tipos de conjuntos de acciones
# sin embargo, en este caso el conjunto de acciones y de estados es discreto.
from stable_baselines3 import PPO
# init agent
model = PPO("MlpPolicy", env, verbose=0)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [8]:
# train the agent and display a progress bar
model.learn(total_timesteps=int(5000), progress_bar=True)

Output()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
model.save("blackjack")

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [10]:
del model
model = PPO.load("blackjack")
model.set_env(env)

In [11]:
from stable_baselines3.common.evaluation import evaluate_policy

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
mean_reward, std_reward

(-0.2, 0.8717797887081348)

El performance del agente con el algoritmo PPO es peor que el aleatorio baseline, la pérdida es mayor pero la desviación estándar respecto a esta es menor lo cual puede ser un buen atributo de lo encontrado.

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [12]:
env.reset()

(array([17,  9,  0]), {})

Vemos que los estados pertenecen a la clase array de python, por lo que procesamos el estado en la función de la misma forma. Nuevamente haciendo flatten() para obtener las dimensiones correctas del array.

In [13]:
import numpy as np

def get_agent_action(model, player_sum, dealer_card, usable_ace):
  state = np.array([player_sum, dealer_card, int(usable_ace)]).flatten()
  action, _ = model.predict(state)
  return action

# Escenario 1
player_sum_1 = 6
dealer_card_1 = 7
usable_ace_1 = False
action_1 = get_agent_action(model, player_sum_1, dealer_card_1, usable_ace_1)
print(f"Escenario 1: Acción = {action_1}")

# Escenario 2
player_sum_2 = 19
dealer_card_2 = 3
usable_ace_2 = True
action_2 = get_agent_action(model, player_sum_2, dealer_card_2, usable_ace_2)
print(f"Escenario 2: Acción = {action_2}")

Escenario 1: Acción = 0
Escenario 2: Acción = 1


Si bien el modelo tiene un valor promedio de ganancia negativo, la respuesta a los casos específicos probados parece ser razonabole y acorde a las reglas del juego dado que con una suma baja toma la acción de pedir más cartas, que es la jugada esperable. Por otro lado, cuando la suma es alta no pide cartas, indicando que su decisión está en cierta medida guiada por el castigo que existe al pasar el 21.

In [14]:
env.close()

### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [15]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

Esto implica que el espacio de decisión es continuo, expandiendo las posibles opciones que el modelo puede considerar.

In [16]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

El ambiente LunarLander describe los estados, acciones y recompensas involucradas en la simulación del aterrizaje de una nave espacial virtual. Las coordenadas del punto objetivo están en (0,0) donde la grilla en la que se puede mover la nave va desde -2.5 a 2.5 tanto en el eje X como Y. Existen otras variables que describen el movimiento de la nave como su velocidad tanto vertical como horizontal y su ángulo de orientación. Contiene también indicadores por cada tren de aterrizaje para saber si están en contacto con el suelo o no.
Los posibles estados son continuos dados por la posición y los otros parámetros descritos anteriormente, que están contenidos en un Box multidimensional condichas características del movimiento de la nave.
El posible espacio de acción es discreto y puede tomar 4 valores, no hacer nada (0), activar el motor izquierdo (1), activar el motor principal (2) o activar el derecho (3).
Se otorgan recompensas en cada paso para guiar el aterrizaje de la nave. La recompensa aumenta a medida que la nave se acerca al punto de aterrizaje y se mueve más lento, y disminuye si la nave está inclinada. Cada tren de aterrizaje que toca el suelo otorga 10 puntos adicionales. Se aplican penalizaciones pequeñas en cada frame cuando los motores laterales están en uso (-0.03 puntos) y una penalización mayor cuando el motor principal se enciende (-0.3 puntos). Al finalizar el episodio, una recompensa adicional de -100 puntos se asigna si la nave choca, y +100 puntos si aterriza exitosamente. Se considera que un episodio está resuelto si la nave obtiene al menos 200 puntos en total.

Las acciones se distinguen de Black Jack principalmente en que hay más acciones que realizar y estas acciones también interactúan más directamente con la recompensa alcanzada, mas allá de qué número aleatorio se asigna o no.

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [17]:
R = []
for episode in range(10):
    obs, info = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        done = terminated or truncated
    R.append(total_reward)
env.close()

print("Promedio de recompensas:", np.mean(R))
print("Desviación de recompensas:", np.std(R))

Promedio de recompensas: -223.18599431399176
Desviación de recompensas: 160.78440737748258


El performance de esta política es muy pobre, de hecho el promedio de las recompensas es exactamente lo contrario a la recompensa necesaria para decir que una iteración se da por terminada, que es 200 puntos. Una política de generación de acciones aleatorias no es adecuada.

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [18]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True)

El modelo A2C soporta el formato Box de

In [19]:
from stable_baselines3 import PPO

model = PPO("MlpPolicy", env, verbose=0)
model.learn(total_timesteps=10000, progress_bar=True)

Output()

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [20]:
model.save("lunar_lander")

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [21]:
del model
model = PPO.load("lunar_lander")
model.set_env(env)

In [22]:
from stable_baselines3.common.evaluation import evaluate_policy

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
mean_reward, std_reward

(-115.4697376, 79.61610107472136)

In [23]:
env.close()

#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [24]:
# Configuración del ambiente
env = gym.make("LunarLander-v3", render_mode="human", continuous=True)

learning_rate = 0.03   # Ajusta según sea necesario
n_steps = 2048           # Número de pasos antes de actualizar
batch_size = 64          # Tamaño del lote para la actualización de gradiente

# Entrenamiento del modelo
model = PPO("MlpPolicy", env, verbose=1, learning_rate=learning_rate, n_steps=n_steps, batch_size=batch_size)
model.learn(total_timesteps=100000, progress_bar=True)  # Aumenta el número de timesteps
model.save("lunar_lander")
del model

# Cargar el modelo y reestablecer el ambiente
model = PPO.load("lunar_lander")
model.set_env(env)

# Evaluación del agente
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=20)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

# Cerrar el ambiente
env.close()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 120      |
|    ep_rew_mean     | -288     |
| time/              |          |
|    fps             | 47       |
|    iterations      | 1        |
|    time_elapsed    | 43       |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 83.1      |
|    ep_rew_mean          | -446      |
| time/                   |           |
|    fps                  | 46        |
|    iterations           | 2         |
|    time_elapsed         | 88        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 84.046196 |
|    clip_fraction        | 0.945     |
|    clip_range           | 0.2       |
|    entropy_loss         | -3.17     |
|    explained_variance   | 8.08e-05  |
|    learning_rate        | 0.03      |
|    loss           

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Mean reward: -611.78407625, Std reward: 183.49704239646204


## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [4]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Enter your Google AI API key: ··········
Enter your Tavily API key: ··········


### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [5]:
%pip install --upgrade --quiet PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.9 MB/s eta 0:00:00


In [7]:
import PyPDF2

doc_paths = ["/content/Dialnet-FundacionDeColoColoEnChile1925-9639350.pdf","/content/El-rol-socio-cultural-del-futbol-en-el-Chile-de-la-segunda-mitad-del-siglo-XX.pdf","/content/Historia_del_Club_Social_y_Deportivo_Colo-Colo.pdf"] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [8]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [9]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.0 MB/s eta 0:00:00


In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [71]:
# Cargar documentos
docs = []
for doc_path in doc_paths:
    loader = PyPDFLoader(doc_path)
    docs.extend(loader.load())

In [12]:
# Dividir en chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

In [13]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00


In [14]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 26.6 MB/s eta 0:00:00


In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS


In [16]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)
vectorstore

#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [19]:
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ace9dae7040>, search_kwargs={'k': 3})

In [20]:
question = "cuando se fundo el club deportivo Colo-Colo?" # pregunta
relevant_documents = retriever.invoke(question) # top k documentos relevantes a la pregunta
relevant_documents

[Document(metadata={'source': '/content/Historia_del_Club_Social_y_Deportivo_Colo-Colo.pdf', 'page': 0}, page_content='institución,4  finalmente optaron por formar un nuevo club con sólidos principios deportivos y morales,\npretensión que fue estipulada en el acta de fundación de Colo-Colo.5  Tras una serie de reuniones, que\ncomenzaron la noche del 12 de abril en la calle Covadonga del barrio Estación Central, la fundación de\nColo-Colo quedó sellada el 19 de abril de 1925 en el Estadio El Llano.6\nEn la primera reunión del club, presidida de forma interina por Juan Quiñones, fueron propuestos varios'),
 Document(metadata={'source': '/content/Historia_del_Club_Social_y_Deportivo_Colo-Colo.pdf', 'page': 1}, page_content='obligatorios, preparación de jugadas y aplicación de tácticas,\nasí como la disponibilidad de implementos y médicos.10 \nLuego de la fundación el equipo se inscribió en la Primera División de la Liga Metropolitana. El primer\npartido que jugó Colo-Colo en su historia y

In [21]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [22]:
retriever_chain = retriever | format_docs # chain
print(retriever_chain.invoke("cuando se fundo el club deportivo Colo-Colo?"))

institución,4  finalmente optaron por formar un nuevo club con sólidos principios deportivos y morales,
pretensión que fue estipulada en el acta de fundación de Colo-Colo.5  Tras una serie de reuniones, que
comenzaron la noche del 12 de abril en la calle Covadonga del barrio Estación Central, la fundación de
Colo-Colo quedó sellada el 19 de abril de 1925 en el Estadio El Llano.6
En la primera reunión del club, presidida de forma interina por Juan Quiñones, fueron propuestos varios

obligatorios, preparación de jugadas y aplicación de tácticas,
así como la disponibilidad de implementos y médicos.10 
Luego de la fundación el equipo se inscribió en la Primera División de la Liga Metropolitana. El primer
partido que jugó Colo-Colo en su historia y en esa división fue ante el English, el 31 de mayo de 1925, y
que terminó con una victoria para Colo-Colo de 6:0. En la misma temporada ganaron su primer "clásico"

futbolístico como en lo financiero. El 23 de enero de 2002 la justicia decretó la

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # modelo de lenguaje
    temperature=0, # probabilidad de "respuestas creativas"
    max_tokens=None, # sin tope de tokens
    timeout=None, # sin timeout
    max_retries=2, # número máximo de intentos
)

llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7ace9dae6d70>, default_metadata=())

In [24]:
from langchain_core.prompts import PromptTemplate


rag_template = '''
Eres un historiador que conoce mucho sobre el club deportivo Colo Colo.
Tu único rol es contestar preguntas del usuario a partir de información relevante que te sea proporcionada.
Responde siempre de la forma más completa posible y usando toda la información entregada.
Responde sólo lo que te pregunten a partir de la información relevante, NUNCA inventes una respuesta.

Información relevante: {context}
Pregunta: {question}
Respuesta útil:
'''

rag_prompt = PromptTemplate.from_template(rag_template)

In [25]:
from langchain_core.output_parsers import StrOutputParser

In [26]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever_chain,
        "question": RunnablePassthrough(),
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

In [27]:
questions = [
    ("¿Cuándo se fundó Colo Colo?"),   # Colo Colo se fundó el 19 de abril de 1925.
     ("¿Quién fue el primer presidente de Colo Colo?"),   # El primer presidente fue David Arellano.
     ("¿Cuál es el estadio de Colo Colo?")   # El estadio de Colo Colo es el Estadio Monumental.
]

for question in questions:

  response = rag_chain.invoke(question)
  result_tuple = (question, response)
  print(result_tuple)



('¿Cuándo se fundó Colo Colo?', 'Colo-Colo fue fundado el 19 de abril de 1925 en el Estadio El Llano.\n')
('¿Quién fue el primer presidente de Colo Colo?', 'La información proporcionada no incluye el nombre del primer presidente de Colo Colo.\n')
('¿Cuál es el estadio de Colo Colo?', 'De acuerdo a la información proporcionada, se menciona el Estadio Monumental, inaugurado definitivamente el 30 de septiembre de 1989.  También se menciona el estadio "Calvo y Bascuñán", pero en el contexto de daños causados por barristas de Colo Colo tras una derrota contra Antofagasta.  No se indica que sea el estadio del club.\n')


Vemos que la solución RAG entrega las repsuestas correctas para 2 de las 3 preguntas, sólamente fallando en una pregunta debido a que, al parecer, la respuesta no se puede encontrar dentro de los documentos entregados al inicio.

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

In [73]:
chunk_sizes = [200, 1000]
for chunk_size in chunk_sizes:
    print(f"\nProbando con chunk size: {chunk_size}")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=50)
    splits = text_splitter.split_documents(docs)

    vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)

    for question in questions:
      response = rag_chain.invoke(question)
      result_tuple = (question, response)
      print(result_tuple)




Probando con chunk size: 200
('¿Cuándo se fundó Colo Colo?', 'Colo-Colo fue fundado el 19 de abril de 1925 en el Estadio El Llano.\n')
('¿Quién fue el primer presidente de Colo Colo?', 'La información proporcionada no incluye el nombre del primer presidente de Colo Colo.\n')
('¿Cuál es el estadio de Colo Colo?', 'De acuerdo a la información proporcionada, se menciona el Estadio Monumental, inaugurado definitivamente el 30 de septiembre de 1989.  También se menciona el estadio "Calvo y Bascuñán", pero en el contexto de daños causados por barristas de Colo Colo tras una derrota contra Antofagasta.  No se indica que sea el estadio del club.\n')
('Cuál es el precio del Bitcoin en pesos Chilenos?', 'La información proporcionada no contiene datos sobre el precio del Bitcoin en pesos chilenos.  Por lo tanto, no puedo responder a tu pregunta.\n')
('Qué equipo ganó la copa Libertadores en 1991?', 'Colo-Colo ganó la Copa Libertadores en 1991.\n')

Probando con chunk size: 1000
('¿Cuándo se fund

A partir de las preguntas, no se observan diferencias en las respuestas generadas variando el tamaño de los chunks.

In [74]:
retrieved_chunks = [2, 10]  # Cantidades de chunks
for k in retrieved_chunks:
    print(f"\nProbando con {k} chunks recuperados")

    retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": retrieved_chunks}, # n° documentos a recuperar
                                     )

    for question in questions:

      response = rag_chain.invoke(question)
      result_tuple = (question, response)
      print(result_tuple)




Probando con 2 chunks recuperados
('¿Cuándo se fundó Colo Colo?', 'Colo-Colo fue fundado el 19 de abril de 1925 en el Estadio El Llano.\n')
('¿Quién fue el primer presidente de Colo Colo?', 'La información proporcionada no indica quién fue el primer presidente de Colo Colo.\n')
('¿Cuál es el estadio de Colo Colo?', 'De acuerdo a la información proporcionada, se menciona el Estadio Monumental, inaugurado definitivamente el 30 de septiembre de 1989.  También se menciona el estadio "Calvo y Bascuñán", pero en el contexto de daños causados por barristas de Colo Colo tras una derrota contra Antofagasta.  No se indica que sea el estadio del club.\n')
('Cuál es el precio del Bitcoin en pesos Chilenos?', 'La información proporcionada no contiene datos sobre el precio del Bitcoin en pesos chilenos.  Por lo tanto, no puedo responder a tu pregunta.\n')
('Qué equipo ganó la copa Libertadores en 1991?', 'Colo-Colo ganó la Copa Libertadores en 1991.\n')

Probando con 10 chunks recuperados
('¿Cuándo

A partir de las preguntas, no se observan diferencias en las respuestas generadas variando el número de chunks seleccionados.

In [82]:
search_types = ["similarity", "mmr"]  # Ejemplo de tipos de búsqueda
for search_type in search_types:
    print(f"\nProbando con tipo de búsqueda: {search_type}")

    retriever = vectorstore.as_retriever(search_type=search_type, # método de búsqueda
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )

    for question in questions:

      response = rag_chain.invoke(question)
      result_tuple = (question, response)
      print(result_tuple)



Probando con tipo de búsqueda: similarity
('¿Cuándo se fundó Colo Colo?', 'Colo-Colo fue fundado el 19 de abril de 1925 en el Estadio El Llano.\n')
('¿Quién fue el primer presidente de Colo Colo?', 'La información proporcionada no indica quién fue el primer presidente de Colo Colo.\n')
('¿Cuál es el estadio de Colo Colo?', 'De acuerdo a la información proporcionada, se menciona el Estadio Monumental, inaugurado definitivamente el 30 de septiembre de 1989.  También se menciona el estadio "Calvo y Bascuñán", pero en el contexto de daños causados por barristas de Colo Colo tras una derrota contra Antofagasta.  No se indica que sea el estadio del club.\n')
('Cuál es el precio del Bitcoin en pesos Chilenos?', 'La información proporcionada no contiene datos sobre el precio del Bitcoin en pesos chilenos.  Por lo tanto, no puedo responder a tu pregunta.\n')
('Qué equipo ganó la copa Libertadores en 1991?', 'Colo-Colo ganó la Copa Libertadores en 1991.\n')

Probando con tipo de búsqueda: mmr
(

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [29]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_search = TavilySearchResults(max_results = 1) # inicializamos tool
tools = [tavily_search] # guardamos las tools en una lista

In [30]:
%pip install --quiet langchain requests


#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [31]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=28ba588c2765042a34f7abef07340c004564dc04920be4b9572bd9fad58df5e2
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [32]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [33]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki_search = WikipediaQueryRun(api_wrapper=api_wrapper)
tools.append(wiki_search)

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

In [34]:
from langchain import hub

react_prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [35]:
from langchain.agents import create_react_agent, AgentExecutor

agent = create_react_agent(llm, tools, react_prompt) # primero inicializamos el agente ReAct
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=50,max_execution_time=60) # lo transformamos a AgentExecutor para habilitar la ejecución de tools
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', 'tool_names': 'tavily_search_results_json, wikipedia'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

In [36]:
response1 = agent_executor.invoke({"input": "Cuál es el precio del Bitcoin en pesos Chilenos?"})
print(response1["output"])

response2 = agent_executor.invoke({"input": "qué equipo ganó la copa Libertadores en 1991?"})
print(response2["output"])



> Entering new AgentExecutor chain...
Thought: I need to find the current price of Bitcoin in Chilean Pesos.  A real-time price is needed, so a search engine is the best option.

Action: tavily_search_results_json
Action Input: "Bitcoin price in Chilean Pesos"
[{'url': 'https://coinmarketcap.com/currencies/bitcoin/btc/clp/', 'content': 'Bitcoin to Chilean Peso Data. The BTC to CLP conversion rate today is $70,203.23. This is a decrease of 0.18% in the last hour and a decrease of 2.89% in the last 24 hours. The recent price direction of Bitcoin is a decrease because BTC is up by 14.58% against CLP in the last 30 days. Our converter updates in real time, giving you accurate'}]Thought: I now have the current Bitcoin price in Chilean Pesos from a reliable source.

Final Answer: El precio actual de Bitcoin en pesos chilenos es aproximadamente $70,203.23.  Tenga en cuenta que este precio puede variar rápidamente.


> Finished chain.
El precio actual de Bitcoin en pesos chilenos es aproxima

A partir de las respuestas podemos ver que el Agente utiliza la tool de Wikipedia cuando para responder la pregunta se necesita de información histórica o más consolidada. Por otro lado, el Agente utiliza la tool de tavily cuando se necesita información más reciente o específica.

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

In [37]:
from langchain.tools import Tool

react_tool = Tool(
    name="ReAct_Agent",
    func=lambda query: agent_executor.invoke({"input": query}),
    description="Un agente que responde preguntas utilizando las tools Wikipedia y Tavily."
)


In [53]:
rag_tool = Tool(
    name="RAG_Solution",
    func=lambda query: rag_chain.invoke({"question": query}),
    description="Una solución basada en RAG que responde preguntas utilizando un modelo de recuperación y generación."
)


#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

In [59]:
tools = [react_tool, rag_tool]

In [68]:
supervisor = create_react_agent(llm, tools, react_prompt)
supervisor_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=100,max_execution_time=120)

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

In [69]:
questions2 = [
    "¿Cuándo se fundó Colo Colo?",
    "¿Quién fue el primer presidente de Colo Colo?",
    "¿Cuál es el estadio de Colo Colo?",
    "Cuál es el precio del Bitcoin en pesos Chilenos?",
    "Qué equipo ganó la copa Libertadores en 1991?"
]

for question in questions2:

  response = supervisor_executor.invoke({"input": question})
  result_tuple = (question, response)
  print(result_tuple)



> Entering new AgentExecutor chain...
Thought: To answer when Colo Colo was founded, I need to consult a reliable source with information on the history of the Chilean football club. Wikipedia is a good starting point.

Action: wikipedia
Action Input: Colo-Colo
wikipedia is not a valid tool, try one of [ReAct_Agent, RAG_Solution].Thought: My previous attempt failed because I tried to use a tool that wasn't available.  I'll try a different approach.  I'll use a general web search to find the founding date of Colo-Colo.  Since I don't have access to `tavily_search_results_json`, I'll assume a general web search will suffice.  I'll need to find a reliable source, such as a reputable sports website or encyclopedia.

Action:  (Simulated Web Search)
Action Input: "Colo-Colo founding date"
(Simulated Web Search) is not a valid tool, try one of [ReAct_Agent, RAG_Solution].Thought: I need to find a way to answer the question without using tools that are unavailable.  I will attempt to answer 

La principal diferencia es que el agente genera respuestas en inglés y, además, las respuestas parecen ser un poco más completas, en el sentido de que entregan la misma información, pero contenidas en una oración más extensa, por ejemplo para la pregunta "¿Qué equipo ganó la copa Libertadores en 1991?, este modelo responde "Colo colo ganó la copa Libertadores en 1991", mientras que el agente anterior solamente respondía "Colo Colo"

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

Este enfoque es más recomendable que la solución Router cuando el usuario requiere responder preguntas basadas en un tema en específico (por ejemplo, documentación empresarial, artículos científicos, etc.) ya que en este caso se necesita un agente capaz de manejar información más específica y contextualizada.Sin embargo, también tiene una desventaja con respecto a Router a la hora de tratar con tareas de ámbitos diferentes, ya que en este caso la solución Router es mejor debido a su capacidad para manejar preguntas de naturaleza diversa.

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [43]:
%pip install --upgrade --quiet gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.2 MB/s eta 0:00:00


Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [81]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = agent_executor.invoke({"input": message})["output"] # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://250ecad1d8bb704850.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
